# PHYS 512 - Final Project

**André Vallières (260742187)**

## Part 0

In [74]:
# Imports
%matplotlib inline
import matplotlib as mpl
from matplotlib import animation
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import time

In [2]:
from base64 import b64encode
from IPython.display import HTML

NBody class for simulating nbody problems. Time evolution follows the leapfrog method. The forces are computed by first defining a kernel which is the potential from a single particle of mass 1. The kernel is then convoluted with the density to arrive at the potential in space, $GM/r$. The accelerations, then, for each particle can be computed by calculating the gradient at each particle's position: $GM/r^2$. Leapfrog is then used to time evolve the system while keeping the total energy bounded.

In [3]:
class NBody:
    def __init__(self, nparticles, m=1.0, v=0.0, pos_init=None, soft=0.03, ngrid=51, G=1.0, dx=0.01, dt=0.01, periodic=True):
        # Parameters
        self.nparticles = nparticles
        self.ngrid = ngrid
        self.soft = soft
        self.G = G
        self.dt = dt
        self.dx = dx
        self.periodic = periodic
        self.lim = np.round(self.ngrid * self.dx / 2.0, 8)
        self.t = 0.0
        
        # Set kernel
        self.kernel = self.get_kernel()
        
        # Convert lists to numpy arrays
        if isinstance(m, list):
            m = np.array(m, dtype=np.float64)
        if isinstance(pos_init, list):
            pos_init = np.array(pos_init, dtype=np.float64)
        if isinstance(v, list):
            v = np.array(v, dtype=np.float64)
            
        # ngrid should be odd to ensure correct position-grid translation
        if self.ngrid % 2 == 0:
            self.ngrid += 1
        
        # Particles
        self.nparticles_init = self.nparticles
        
        if pos_init is not None:
            assert(pos_init.shape[1] == nparticles)
            self.pos = pos_init
        else:
            self.pos = self.lim * (2 * np.random.rand(3, nparticles) - 1)
            #self.pos = self.get_positions((ngrid//3) * (np.random.randn(3, nparticles) + ngrid//2) % (ngrid - 1))
        self.pos_init = self.pos.copy()
        
        if isinstance(m, np.ndarray):
            assert(m.shape[1] == nparticles)
            self.masses = m
        else:
            self.masses = m * np.ones((1, nparticles))
        self.masses_init = self.masses.copy()
            
        if isinstance(v, np.ndarray):
            assert(v.shape[1] == nparticles)
            self.vels = v
        else:
            self.vels = v * np.ones((3, nparticles))
        self.vels_init = self.vels.copy()
        
        # Get accelerations
        self.accls = self.get_accls(self.pos)

    def get_kernel_n(self, n):
        # Set up the grid
        x = np.arange(-n//2 + 1, n//2 + 1)
        x = np.abs(x) * self.dx
        xx,yy,zz = np.meshgrid(x, x, x)

        # Set up kernel
        r = np.sqrt(xx**2 + yy**2 + zz**2)
        r[r < self.soft] = self.soft
        kernel = -self.G / r

        return kernel
        
    def get_kernel(self):
        if hasattr(self, 'kernel'):
            return self.kernel
        
        # Set up the grid
        x = np.arange(-self.ngrid//2 + 1, self.ngrid//2 + 1)
        x = np.abs(x) * self.dx
        xx,yy,zz = np.meshgrid(x, x, x)

        # Set up kernel
        r = np.sqrt(xx**2 + yy**2 + zz**2)
        r[r < self.soft] = self.soft
        self.kernel = -self.G / r

        return self.kernel
    
    def get_potential(self):
        self.rho = self.get_density()
        
        if self.periodic:
            rhoft = np.fft.fftn(self.rho)
            return np.fft.ifftn(rhoft * np.fft.fftn(np.fft.fftshift(self.kernel))).real
        else:
            # First pad kernel and density (effectively increasing space for a moment)
            p = self.ngrid//2
            rho = np.pad(self.rho, ((p,p),(p,p),(p,p)))
            kernel = self.get_kernel_n(rho.shape[0])

            # Then perform convolution as usual
            rhoft = np.fft.fftn(rho)
            pot = np.fft.ifftn(rhoft * np.fft.fftn(np.fft.fftshift(kernel))).real
            
            # Keep only the initial space
            pot = pot[p:-p, p:-p, p:-p]
            
            return pot
    
    def get_density(self):
        # Iterate through particles to add their mass
        rho = np.zeros((self.ngrid,self.ngrid,self.ngrid))
        
        for i, mass in enumerate(self.masses[0]):
            idxs = self.get_indices(self.pos[:,i])
            
            # TODO: Could do something fancier like giving
            # weights depending on the distance to the position
            
            # x is left, right (l, r)
            # y is down, up (d, u)
            # z is front, back (f, b)
            if self.periodic:
                l = tuple((idxs - [1,0,0]) % self.ngrid)
                r = tuple((idxs + [1,0,0]) % self.ngrid)
                d = tuple((idxs - [0,1,0]) % self.ngrid)
                u = tuple((idxs + [0,1,0]) % self.ngrid)
                f = tuple((idxs - [0,0,1]) % self.ngrid)
                b = tuple((idxs + [0,0,1]) % self.ngrid)
            
            else:
                l = (idxs - [1,0,0])
                r = (idxs + [1,0,0])
                d = (idxs - [0,1,0])
                u = (idxs + [0,1,0])
                f = (idxs - [0,0,1])
                b = (idxs + [0,0,1])
                
                if (l < 0).any() or (l >= self.ngrid).any():
                    l = idxs
                if (r < 0).any() or (r >= self.ngrid).any():
                    r = idxs
                if (d < 0).any() or (d >= self.ngrid).any():
                    d = idxs
                if (u < 0).any() or (u >= self.ngrid).any():
                    u = idxs
                if (f < 0).any() or (f >= self.ngrid).any():
                    f = idxs
                if (b < 0).any() or (b >= self.ngrid).any():
                    b = idxs
                    
                l = tuple(l)
                r = tuple(r)
                d = tuple(d)
                u = tuple(u)
                f = tuple(f)
                b = tuple(b)
            
            # Give 1/12 of the mass to each neighbors
            rho[l] += mass / 12.0
            rho[r] += mass / 12.0
            rho[d] += mass / 12.0
            rho[u] += mass / 12.0
            rho[f] += mass / 12.0
            rho[b] += mass / 12.0
                
            # Keep 1/2 in the box itself
            rho[tuple(idxs)] += mass / 2.0
            
        return rho
        
    def get_accls(self, pos):
        # Convolution between density and kernel
        self.pot = self.get_potential()
        
        # Force is (negative) gradient of potential
        # We don't multiply by rho (mass), since we want the acceleration later
        ax = -(np.roll(self.pot, -1, axis=0) - np.roll(self.pot, 1, axis=0)) / (2 * self.dx) 
        ay = -(np.roll(self.pot, -1, axis=1) - np.roll(self.pot, 1, axis=1)) / (2 * self.dx)
        az = -(np.roll(self.pot, -1, axis=2) - np.roll(self.pot, 1, axis=2)) / (2 * self.dx)
        
        if not self.periodic:
            # If not periodic, set gradient to 0 at the edges
            for a in [ax, ay, az]:
                a[0,:,:] = 0.0
                a[self.ngrid - 1,:,:] = 0.0
                
                a[:,0,:] = 0.0
                a[:,self.ngrid - 1,:] = 0.0
                
                a[:,:,0] = 0.0
                a[:,:,self.ngrid - 1] = 0.0
        
        # Get particles positions
        idxs = self.get_indices(pos)
        
        ax = ax[idxs[0,:],idxs[1,:],idxs[2,:]]
        ay = ay[idxs[0,:],idxs[1,:],idxs[2,:]]
        az = az[idxs[0,:],idxs[1,:],idxs[2,:]]
        
        return np.array([ax,ay,az])

    def get_indices(self, pos):
        idxs = np.rint(pos / self.dx + (self.ngrid - 1)/2).astype(int)
        return idxs
    
    def get_positions(self, idxs):
        pos = np.round((idxs - (self.ngrid - 1)/2) * self.dx, 8)
        return pos
    
    def get_energy(self):
        ke = 0.5 * np.sum(self.masses * self.vels**2)
    
        pe = 0
        for i in range(self.nparticles):
            # Get particle position
            pos = self.pos[:,i]
            
            # Get indices
            idxs = self.get_indices(pos)
            
            # Get potential at that location and add
            p = self.masses[0, i] * self.pot[idxs[0], idxs[1], idxs[2]]
            pe += p
            
        return ke, pe
    
    def apply_boundary_conditions(self):
        if self.periodic:
            # Wrap around for each axis
            lim = np.round(self.ngrid * self.dx / 2.0, 8)
            for i in range(3):
                for j in range(self.nparticles):
                    while self.pos[i,j] > lim:
                        self.pos[i,j] -= 2*lim
                    while self.pos[i,j] < -lim:
                        self.pos[i,j] += 2*lim
        else:
            # Remove particles that go outside of grid
            lim = np.round(self.ngrid * self.dx / 2.0, 8)
            
            to_be_removed = []
            for i in range(self.nparticles):
                pos = self.pos[:,i]
                
                if (pos > lim).any() or (pos < lim).any():
                    to_be_removed.append(i)
                    
            # Remove particle
            self.vels = np.delete(self.vels, to_be_removed, 1)
            self.accls = np.delete(self.accls, to_be_removed, 1)
            self.pos = np.delete(self.pos, to_be_removed, 1)
            self.masses = np.delete(self.masses, to_be_removed, 1)
            
            self.nparticles = self.vels.shape[1]
                
            self.rho = self.get_density()
            self.pot = self.get_potential()
    
    def evolve(self):
        # Update velocities
        self.vels += self.accls * self.dt / 2.0
        
        # Update positions
        self.pos += self.vels * self.dt
        
        # Get accelerations
        try:
            self.accls = self.get_accls(self.pos)
        except:
            # Fix for boundary conditions (ugly)
            lim = np.round(self.ngrid * self.dx / 2.0, 8)
            for i in range(3):
                for j in range(self.nparticles):
                    while self.pos[i,j] > lim:
                        self.pos[i,j] -= 2*lim
                    while self.pos[i,j] < -lim:
                        self.pos[i,j] += 2*lim
                        
            self.accls = self.get_accls(self.pos)
        
        # Update velocities
        self.vels += self.accls * self.dt / 2.0
        
        # Update time
        self.t += self.dt
        
    def reset(self):
        # Reset positions, velocities, and accelerations
        self.pos = self.pos_init.copy()
        self.vels = self.vels_init.copy()
        self.accls = self.get_accls(self.pos)
        
        # Reset masses and number of particles
        self.masses = self.masses_init.copy()
        self.nparticles = self.nparticles_init
        
        # Reset time
        self.t = 0
        
        # Update other parameters
        self.rho = self.get_density()
        self.pot = self.get_potential()
        

Useful functions for visualization.

In [ ]:
def visualize_space(nbody, fig=None, ax=None, plot=None, figsize=(8,8), title=None, 
                    plot_type="2d_scatter", cmap=cm.viridis, cb=None, history=False,
                    marker_weight=10):
    if not fig and not ax:
        if "2d" in plot_type:
            fig, ax = plt.subplots(1, 1, figsize=figsize)
        elif "3d" in plot_type:
            fig = plt.figure(figsize=figsize)
            ax = fig.add_subplot(1, 1, 1, projection='3d')
        else:
            raise ValueError("Unexpected value of plot_type keyword argument")
        
    lim = np.round(nbody.ngrid * nbody.dx / 2.0, 8)
    
    if "scatter" in plot_type:
        space = nbody.pos
    elif "density" in plot_type:
        space = nbody.rho
    elif "potential" in plot_type:
        space = nbody.pot
    else:
        raise ValueError("Unexpected value of plot_type keyword argument")
    
    if "2d" in plot_type:
        if "scatter" in plot_type:
            if history or not plot:
                plot = ax.scatter(space[0,:], space[1,:], color='b', s=marker_weight)
            else:
                plot.set_offsets(space[0:2,:].T)
            
            ax.set_xlim((-lim,lim))
            ax.set_ylim((-lim,lim))
        elif "density" in plot_type:
            plot = ax.imshow(space.sum(axis=2), cmap=cmap, extent=[-lim,lim,-lim,lim], origin="lower")
            
            if cb is not None:
                cb.remove()
                plt.draw()
            cb = fig.colorbar(plot)
            
    elif "3d" in plot_type:
        if "scatter" in plot_type:
            if history or not plot:
                plot = ax.scatter(space[0,:], space[1,:], space[2,:], color='b', s=marker_weight)
            else:
                plot.set_offsets(space.T)
            
            ax.set_xlim3d((-lim,lim))
            ax.set_ylim3d((-lim,lim))
            ax.set_zlim3d((-lim,lim))
            
            ax.set_zlabel(r"$z$", fontsize=10)
        elif "density" in plot_type:
            pass
            # TODO
            
        #ax.clear()
        #surf = ax.plot_surface(X, Y, prob, rstride=5, cstride=5, cmap=cmap, alpha=0.7, norm=mpl.colors.Normalize(-wlim, wlim))
        #if colorbar:
        #    if cb is not None:
        #        cb.remove()
        #        plt.draw()
        #    cb = fig.colorbar(surf)
    else:
        raise ValueError("Unexpected value of plot_type keyword argument")
    
    ax.set_title(title)
    
    ax.set_xlabel(r"$x$", fontsize=10)
    ax.set_ylabel(r"$y$", fontsize=10)
    
    return fig, ax, cb, plot

cb = None 
plot = None
def evolve_space(nbody, animation_name, plot_type="2d_scatter", iters=50, overs=5, figsize=(8,8), marker_weight=10, history=False, fps=10):
    # Animate
    if "2d" in plot_type:
        fig, ax = plt.subplots(1, 1, figsize=figsize)
    elif "3d" in plot_type:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(1, 1, 1, projection='3d')
    else:
        raise ValueError("Unexpected value of plot_type keyword argument")
    
    global plot
    plot = None
    
    # Record start time
    start_time = time.time()

    def plot_and_update_space(n, fig=None, ax=None):
        global cb, plot
        ke, pe = nbody.get_energy()
        fig, ax, cb, plot = visualize_space(
            nbody, 
            title=r"$t = %f$s"%nbody.t + "\n" + r"$E = %.3f$" % (ke + pe),
            fig=fig, 
            ax=ax, 
            plot=plot,
            cb=cb,
            figsize=figsize,
            history=history,
            marker_weight=marker_weight,
            plot_type=plot_type)
        
        for _ in range(overs):
            nbody.evolve()
        return fig, ax
     
    anim = animation.FuncAnimation(fig, plot_and_update_space, frames=iters, fargs=(fig, ax,))
    anim.save(animation_name + ".mp4", writer=animation.FFMpegWriter(fps=fps))
    plt.close(fig)
    
    # Print elapsed time
    elapsed_time = time.time() - start_time
    print("Simulation took: " + time.strftime("%Hh:%Mm:%Ss", time.gmtime(elapsed_time)))

def display_embedded_video(filename):
    video = open(filename, "rb").read()
    video_encoded = b64encode(video).decode("ascii")
    video_tag = '<video controls alt="test" src="data:video/x-m4v;base64,{0}">'.format(video_encoded)
    return HTML(video_tag)

## Part 1

Let's put a single particle in the grid.

In [61]:
G = 1e6
dx = 0.025
soft = 4 * dx
dt = 0.001
ngrid = 101

nbody = NBody(
    1, 
    ngrid=ngrid, 
    dx=dx, 
    soft=soft, 
    dt=dt, 
    pos_init=[[0],[0],[0]], 
    G=G
)

Let it now evolve and make sure position hasn't changed and kinetic energy is null, even with G insanely high.

In [38]:
for _ in range(100):
    nbody.evolve()
print("Position:", nbody.pos[:,0])
print("Kinetic energy:", nbody.get_energy()[0])

Position: [0.00000000e+00 4.24701720e-10 8.49403441e-10]
Kinetic energy: 7.910685995149329e-17


And we can also let it evolve and look at the movie.

In [29]:
G = 1e-2

nbody = NBody(
    1, 
    ngrid=ngrid, 
    dx=dx, 
    soft=soft, 
    dt=dt, 
    pos_init=[[0],[0],[0]], 
    v=[[0],[0],[0]],
    G=G
)

evolve_space(
    nbody, 
    "part1_single_particle",
    iters=50,
    history=True
)

# and 3d
evolve_space(
    nbody, 
    "part1_single_particle_3d",
    plot_type="3d_scatter",
    iters=50,
    history=True
)

In [89]:
display_embedded_video("part1_single_particle.mp4")

In [30]:
display_embedded_video("part1_single_particle_3d.mp4")

## Part 2

Now we put two particles in our grid.

To find the optimal starting velocity, I've used this code below (ran outside of this notebook). It basically scans through various velocities (close to the expected value) and find the velocity that gives the smallest absolute deviation.

In [ ]:
M = 10000
m = 1
r = 0.3

v = np.sqrt(G * (M + m) / r)

abs_diff_arr = []
v_arr = np.linspace(v-2.5, v+2.5, 100)

for v in v_arr:
    nbody = NBody(
        2, 
        ngrid=ngrid, 
        dx=dx, 
        soft=soft, 
        dt=dt, 
        m=[[M, m]],
        pos_init=[[0, r],[0, 0],[0, 0]], 
        v=[[0, 0],[0, v],[0, 0]],
        G=G
    )
    
    abs_diff = 0
    for _ in range(50):
        nbody.evolve()
        d = (np.sqrt(np.sum((nbody.pos[:,1] - nbody.pos[:,0])**2)) - r)
        abs_diff += np.abs(d)
        
    abs_diff_arr.append(abs_diff)
    
v_arr = np.array(v_arr)
abs_diff_arr = np.array(abs_diff_arr)

print(v_arr)
print(abs_diff_arr)

print("Optimal")
best = np.argmin(abs_diff_arr)
print(v_arr[best])
print(abs_diff_arr[best])

Now let's plot the result.

In [36]:
v = 16.580553653836304

nbody = NBody(
        2, 
        ngrid=ngrid, 
        dx=dx, 
        soft=soft, 
        dt=dt, 
        m=[[M, m]],
        pos_init=[[0, r],[0, 0],[0, 0]], 
        v=[[0, 0],[0, v],[0, 0]],
        G=G
    )

evolve_space(
    nbody, 
    "part2_two_particles",
    iters=150,
    history=True
)

# and 3d
evolve_space(
    nbody, 
    "part2_two_particles_3d",
    plot_type="3d_scatter",
    iters=150,
    history=True
)

In [38]:
display_embedded_video("part2_two_particles.mp4")

In [37]:
display_embedded_video("part2_two_particles_3d.mp4")

Note that the 3D plot is the continuation of the 2D plot (by looking at the time) and the orbit stays circular.

## Part 3

### Periodic boundaries

In [77]:
G = 0.0001
dx = 5.0/200
soft = 4 * dx
dt = 0.01
nparticles = 10000
ngrid = 201
m = 5

nbody = NBody(
        nparticles, 
        ngrid=ngrid,
        m=m,
        dx=dx, 
        soft=soft, 
        dt=dt, 
        G=G
    )

In [78]:
nbody.reset()
evolve_space(
    nbody, 
    "part3_periodic_3d",
    plot_type="3d_scatter",
    marker_weight=0.2,
    iters=200
)

Simulation took: 00h:22m:36s


In [79]:
display_embedded_video("part3_periodic_3d.mp4")

In [72]:
G = 1
dx = 0.025
soft = 4 * dx
dt = 0.01
nparticles = 100000
ngrid = 101
m = 100

nbody2 = NBody(
        nparticles, 
        ngrid=ngrid, 
        dx=dx, 
        m=m,
        soft=soft, 
        dt=dt, 
        G=G
    )

In [73]:
nbody2.reset()
evolve_space(
    nbody2, 
    "part3_periodic_3d-nbody2",
    plot_type="3d_scatter",
    marker_weight=0.2,
    iters=200
)

### Non-periodic boundaries

Now, before doing the same thing for non-periodic boundaries, we can plut the collapsed potential and see how the boundaries are non-periodic.

In [76]:
nbody = NBody(
    1, 
    ngrid=ngrid, 
    dx=dx, 
    soft=soft, 
    dt=dt, 
    pos_init=[[1.0],[0.0],[0.0]], 
    G=G,
    periodic=False
)

visualize_space(nbody, plot_type="2d_potential")
plt.show()

TypeError: __init__() got an unexpected keyword argument 'periodic'

## Part 4